## Setup directories

In [20]:
import os
from skimage.io import imread,imsave
from prepare_run import *
import numpy as np
import shutil
import stat
import itertools
import re
import random
import time
from pathlib import Path
from collections import defaultdict
from fastprogress.fastprogress import progress_bar,master_bar
from tqdm import tqdm


In [21]:

# explicit_ranges = {4:range(0,100),5:range(0,100),6:range(0,200),11:range(0,220),12:range(0,110),13:range(0,125)}; #random2
explicit_ranges = {}; #migration6
stage_ranges = defaultdict(lambda: None,explicit_ranges)
# stage_ranges[10] = range(0,1);
# stage_ranges[11]= range(0,1);
filename_regex = 'p[0-9]*_s([0-9]+)_t([0-9]+).*\.(TIF|TIFF|tif|tiff)';

moviePath = Path("G:/Other computers/USB and External Devices/USB_DEVICE_1643752484/2022.3.1 Migration Test 6");

outImages = processFolder/"segmentation_images";
completeMasks = processFolder/"segmentation_output_masks";
maskSuffix = "";#"_mask";

moviesProcessingFolder = iteration_testing_folder/"movie_segmentation/";
runOutputFolder = moviesProcessingFolder/os.path.basename(moviePath);
testImages = runOutputFolder/"test_images"; ##where the images used in the test segmentation will be stored
testOutput = runOutputFolder/"test_segmentation" ##where the masks will go for the test segmentaiton
fullOutput = runOutputFolder/"full_segmentation"; ##where the masks will go for the full movie


if not os.path.exists(runOutputFolder): os.mkdir(runOutputFolder);
if not os.path.exists(testOutput): os.mkdir(testOutput);
if not os.path.exists(fullOutput): os.mkdir(fullOutput);
if not os.path.exists(testImages): os.mkdir(testImages);

def on_rm_error( func, path, exc_info):
    # path contains the path of the file that couldn't be removed
    # let's just assume that it's read-only and unlink it.
    os.chmod( path, stat.S_IWRITE )
    # os.unlink( path )

In [22]:
### GCP Parameters
bucket = "optotaxisbucket" #@param {type:"string"}

GCP_parent_dir = "movie_segmentation" #@param {type:"string"}
GCP_parent_dir = Path(bucket)/GCP_parent_dir

In [23]:
##check if processing directories empty:
if (any(os.scandir(outImages)) != 0):
    while True:
        doDelete = input(f"Warning: image processing directory {outImages}\nmust be empty; delete? (y/n), \'cancel\' to cancel\n",);
        if doDelete == "y":
            for im in os.scandir(outImages):
                f = os.path.join(outImages,im)
                if os.path.isdir(f): shutil.rmtree(f,onerror = on_rm_error ); #just in case
                else: os.remove(f);
            break;
        elif doDelete == "n":
            break;
        elif doDelete.lower() == "cancel":
            exit();
    
if (any(os.scandir(completeMasks)) != 0):
    while True:
        doDelete = input(f"Warning: mask output directory {completeMasks}\nmust be empty; delete?  (y/n), \'cancel\' to cancel\n",);
        if doDelete == "y":
            print("deleting");
            for im in os.scandir(completeMasks):
                f = os.path.join(completeMasks,im)
                if os.path.isdir(f): shutil.rmtree(f,onerror = on_rm_error ); #just in case
                else: os.remove(f);
            break;
        elif doDelete == "n":
            break;
        elif doDelete.lower() == "cancel":
            exit();

deleting


## Test Segmentation

In [24]:
##check if processing directories empty:
if (any(os.scandir(testImages)) != 0):
    while True:
        doDelete = input(f"Warning: image input directory {testImages}\nmust be empty; delete? (y/n), \'cancel\' to cancel\n",);
        # print("inputted??")
        if doDelete == "y":
            for im in os.scandir(testImages):
                f = os.path.join(testImages,im)
                if os.path.isdir(f): shutil.rmtree(f,onerror = on_rm_error ); #just in case
                else: os.remove(f);
            break;
        elif doDelete == "n":
            break;
        elif doDelete.lower() == "cancel":
            exit();
    
if (any(os.scandir(testOutput)) != 0):
    while True:
        doDelete = input(f"Warning: mask output directory {testOutput}\nmust be empty; delete?  (y/n), \'cancel\' to cancel\n",);
        if doDelete == "y":
            for im in os.scandir(testOutput):
                try: shutil.rmtree(os.path.join(testOutput,im),onerror = on_rm_error ); #just in case
                except OSError: os.remove(os.path.join(testOutput,im));
            break;
        elif doDelete == "n":
            break;
        elif doDelete.lower() == "cancel":
            exit();

In [25]:
##copy random test images (n from each stage position) into the test folder
images_per_position = 2;
names = os.listdir(moviePath);

def stage_from_name(name:str):
    m = re.match(filename_regex,name);
    return m.group(1) if m else "-1";

#groupby: itertools function that splits a list into sublists based on the value of a key function
for k,g in itertools.groupby(names,stage_from_name):
    g = list(g)
    # print(g)
    if k == "-1": continue;
    for n in random.sample(g,min(images_per_position,len(g))):
        shutil.copyfile(moviePath/n,testImages/n);
    

In [26]:
##copy and split images from input directory to processing directory
names = os.listdir(testImages);
print(len(names));
maskNames = [];
for name in tqdm(names):
    im = imread(testImages/name);
    # print(name);
    if auto_rescale:
        im = rescale_intensity(im);
    assert isinstance(im,np.ndarray);
    sliced = False
    if x_slices > 1 or y_slices > 1:
        M = (im.shape[0]-context_bounds[0]-context_bounds[2]-crop[0]-crop[2])/y_slices;
        N = (im.shape[1]-context_bounds[1]-context_bounds[3]-crop[1]-crop[3])/x_slices;

        if int(M) != M or int(N) != N:
            raise Exception(f"ERROR: Mask with size {im.shape[:2]} cannot be sliced into {x_slices} columns and {y_slices} rows\nwith context bounds of {context_bounds}; {M} and {N} not integers");
        else:
            M = int(M)
            N = int(N)
            im = (im/256).astype('uint8');
            im = np.stack((im,im,im),axis=2);
            sliced = True;
            tiles = [im[y-context_bounds[0]:y+M+context_bounds[2],x-context_bounds[1]:x+N+context_bounds[3]] 
                    for y in range(context_bounds[0]+crop[0],im.shape[0]-crop[0]-crop[2]-context_bounds[0]-context_bounds[2],M) 
                    for x in range(context_bounds[1]+crop[1],im.shape[1]-crop[1]-crop[3]-context_bounds[1]-context_bounds[3],N)];
            imBounds = [[y-context_bounds[0],y+M+context_bounds[2],x-context_bounds[1],x+N+context_bounds[3]] 
                    for y in range(context_bounds[0]+crop[0],im.shape[0]-crop[0]-crop[2]-context_bounds[0]-context_bounds[2],M) 
                    for x in range(context_bounds[1]+crop[1],im.shape[1]-crop[1]-crop[3]-context_bounds[1]-context_bounds[3],N)];

            outMasks = [name,[]];
            for num,m in enumerate(tiles):
                outMasks[1].append(os.path.splitext(name)[0] + f"-{num}" + ".TIF");
                imsave(outImages/(os.path.splitext(name)[0] + f"-{num}" + ".TIF"), m,check_contrast=False)
            maskNames.append(outMasks.copy());
    else:   
        imsave(outImages+os.path.splitext(name)[0] + ".TIF", im,check_contrast=False)
print("image splitting complete");

32


100%|██████████| 32/32 [00:06<00:00,  5.12it/s]

image splitting complete


### Begin segmentation on remote hardware
if set-up correctly, google drive will start syncing files to the cloud while the above cell is processing. When you see "image splitting complete", check that google drive has finished syncing, then begin the sementation

### Check for segmentation complete
This cell will constantly check the remote segmentation output directory for a flag file saying segmentation is complete. When the cell finishes executing, run the next cell to stitch together the output.

In [27]:
#Check for complete segmentation; skip this step if segmentation is complete and drive desktop sync is all up to date
while True:
    if os.path.exists(completeMasks/'segmentation_complete.flag'):
        break;
    time.sleep(60);
print("Segmentation file flag found; waiting for sync to complete");
time.sleep(120); ##make sure everything else finishes syncing too

Segmentation file flag found; waiting for sync to complete


### Stitch test segmentation output
Run cell only if above cell has finished executing or if remote segmentation has completed successfully and drive has **fully** synced. 

In [28]:
errored = False;
for n in tqdm(maskNames):
    try:
        if isinstance(n,list):
            names = n[1];
            baseName = n[0];
            stitchMasks = [imread(completeMasks/(os.path.splitext(name)[0] + maskSuffix + os.path.splitext(name)[1])) for name in names];
            for i,m in enumerate(stitchMasks):
                x = i // y_slices;
                y = i % x_slices;
                imBounds = [crop[0]+context_bounds[0] if x != 0 else 0,m.shape[0]-crop[2]-context_bounds[2] if x != x_slices-1 else m.shape[0],crop[1]+context_bounds[1] if y!= 0 else 0 ,m.shape[1]-crop[3]-context_bounds[3] if y != y_slices - 1 else m.shape[1]];
                stitchMasks[i] = m[imBounds[0]:imBounds[1],imBounds[2]:imBounds[3]];
            stitched = np.concatenate([np.concatenate(stitchMasks[i*x_slices:(i+1)*x_slices],axis=1) for i in range(y_slices)]);
            # print(stitched.shape);
            imsave(testOutput/(os.path.splitext(baseName)[0]+".TIF"),stitched,check_contrast=False);
    except Exception as e:
        print(f"error: {n[0]} does not exist")
        print(e);
        errored = True;
if not errored and not os.path.exists(completeMasks/'stitching_complete.flag'):
    with open(completeMasks/'stitching_complete.flag','w') as f:
        pass;

100%|██████████| 32/32 [00:06<00:00,  5.17it/s]


### Clear temporary processing directories
Make sure the test segmentation was complete before running this cell, as if files have not been properly stitched and copied to movie_segmentation/{movie_name}/test_segmentation, clearing these directories will lose that data.

In [10]:
if not errored and os.path.exists(completeMasks/"stitching_complete.flag"):# Clear temporary directories
    for im in os.scandir(outImages):
        try: shutil.rmtree(os.path.join(outImages,im),onerror = on_rm_error ); #just in case
        except OSError: os.remove(os.path.join(outImages,im));
    for im in os.scandir(completeMasks):
        try: shutil.rmtree(os.path.join(completeMasks,im),onerror = on_rm_error ); #just in case
        except OSError: os.remove(os.path.join(completeMasks,im));
else:
    print("warning: stitching was not complete. Please ensure stitching worked correctly before removing temporary directories");

## Full Segmentation - GCP + smart stitching detection

### Check if the full movie output directory is empty
If this directory is not empty, masks might be overwritten, and extra masks might be left in. Please copy any files you want to keep and then clear the directory.

In [24]:
##check if output directories empty:
if (any(os.scandir(fullOutput)) != 0):
    while True:
        doDelete = input(f"Warning: image processing directory {fullOutput}\nmust be empty; delete? (y/n), \'cancel\' to cancel\n",);
        if doDelete == "y":
            for im in os.scandir(fullOutput):
                try: shutil.rmtree(os.path.join(fullOutput,im),onerror = on_rm_error ); #just in case
                except OSError: os.remove(os.path.join(fullOutput,im));
            break;
        elif doDelete == "n":
            break;
        elif doDelete.lower() == "cancel":
            exit();

### Copy and split images from input directory to processing directory
GCP doesn't care about folder sizes so all images will go into one directory and be uploaded together

In [25]:
##copy and split images from input directory to processing directory
names = os.listdir(moviePath);
for name in tqdm(names):
    if name.endswith('.nd'):
        print(f"ignoring .nd file");
        continue;

    try:
        im = imread(moviePath/name);
    except Exception as e:
        print(e);
        continue;
    
    print(name);
    if auto_rescale:
        im = rescale_intensity(im);
    assert isinstance(im,np.ndarray);
    
    # print(im.shape);
    M = (im.shape[0]-context_bounds[0]-context_bounds[2]-crop[0]-crop[2])/y_slices;
    N = (im.shape[1]-context_bounds[1]-context_bounds[3]-crop[1]-crop[3])/x_slices;
    # print(M,N);
    # print(-context_bounds[1]-context_bounds[3]-crop[1]-crop[3])
    # print(im.shape);

    if int(M) != M or int(N) != N:
        raise Exception(f"ERROR: Mask with size {im.shape[:2]} cannot be sliced into {x_slices} columns and {y_slices} rows\nwith context bounds of {context_bounds}; {M} and {N} not integers");
    else:
        M = int(M)
        N = int(N)
        im = (im/256).astype('uint8');
        im = np.stack((im,im,im),axis=2);
        tiles = [im[y-context_bounds[0]:y+M+context_bounds[2],x-context_bounds[1]:x+N+context_bounds[3]] 
                for y in range(context_bounds[0]+crop[0],im.shape[0]-crop[0]-crop[2]-context_bounds[0]-context_bounds[2],M) 
                for x in range(context_bounds[1]+crop[1],im.shape[1]-crop[1]-crop[3]-context_bounds[1]-context_bounds[3],N)];
        imBounds = [[y-context_bounds[0],y+M+context_bounds[2],x-context_bounds[1],x+N+context_bounds[3]] 
                for y in range(context_bounds[0]+crop[0],im.shape[0]-crop[0]-crop[2]-context_bounds[0]-context_bounds[2],M) 
                for x in range(context_bounds[1]+crop[1],im.shape[1]-crop[1]-crop[3]-context_bounds[1]-context_bounds[3],N)];

        # print((context_bounds[0]+crop[0],context_bounds[1]+crop[1]))
        # print(imBounds);
        for num,m in enumerate(tiles):
            imsave(outImages/(os.path.splitext(name)[0] + f"-{num}" + ".TIF"), m,check_contrast=False)
print("image transfer complete");

ignoring .nd file
p1_s10_t1.TIF
p1_s10_t10.TIF
p1_s10_t100.TIF
p1_s10_t101.TIF
p1_s10_t102.TIF
p1_s10_t103.TIF
p1_s10_t104.TIF
p1_s10_t105.TIF
p1_s10_t106.TIF
p1_s10_t107.TIF
p1_s10_t108.TIF
p1_s10_t109.TIF
p1_s10_t11.TIF
p1_s10_t110.TIF
p1_s10_t111.TIF
p1_s10_t112.TIF
p1_s10_t113.TIF
p1_s10_t114.TIF
p1_s10_t115.TIF
p1_s10_t116.TIF
p1_s10_t117.TIF
p1_s10_t118.TIF
p1_s10_t119.TIF
p1_s10_t12.TIF
p1_s10_t120.TIF
p1_s10_t121.TIF
p1_s10_t122.TIF
p1_s10_t123.TIF
p1_s10_t124.TIF
p1_s10_t125.TIF
p1_s10_t126.TIF
p1_s10_t127.TIF
p1_s10_t128.TIF
p1_s10_t129.TIF
p1_s10_t13.TIF
p1_s10_t130.TIF
p1_s10_t131.TIF
p1_s10_t132.TIF
p1_s10_t133.TIF
p1_s10_t134.TIF
p1_s10_t135.TIF
p1_s10_t136.TIF
p1_s10_t137.TIF
p1_s10_t138.TIF
p1_s10_t139.TIF
p1_s10_t14.TIF
p1_s10_t140.TIF
p1_s10_t141.TIF
p1_s10_t142.TIF
p1_s10_t143.TIF
p1_s10_t144.TIF
p1_s10_t145.TIF
p1_s10_t146.TIF
p1_s10_t147.TIF
p1_s10_t148.TIF
p1_s10_t149.TIF
p1_s10_t15.TIF
p1_s10_t150.TIF
p1_s10_t151.TIF
p1_s10_t152.TIF
p1_s10_t153.TIF
p1_s10_t154.TI

### Copy files to GCP
If you don't have the credentials to upload to GCP automatically, please copy the files from \segmentation_iteration_testing\processing\segmentation_images to the GCP Bucket's movie_segmentation/input_images/ directory

In [30]:
print(os.system(f'gsutil -m rm -o GSUtil:parallel_process_count=1 -r "gs://{GCP_parent_dir/os.path.basename(outImages)}"'))
print(os.system(f'gsutil -m cp -o GSUtil:parallel_process_count=1 -r "{outImages}" "gs://{GCP_parent_dir}"'));
# print(outImages)
# print(GCP_parent_dir);

C:\Users\Harrison Truscott\OneDrive - University of North Carolina at Chapel Hill\Bear Lab\optotaxis calibration\data\segmentation_iteration_testing\processing\segmentation_images
optotaxisbucket\movie_segmentation


In [6]:
#Remove local image files
for im in tqdm(os.listdir(outImages)):
    f = os.path.join(outImages,im)
    if os.path.isdir(f): shutil.rmtree(f,onerror = on_rm_error ); #just in case
    else: os.remove(f);

0it [00:00, ?it/s]


### Wait for remote segmentation to complete
This cell will constantly check each stage folder for a flag file from the remote segmentation program that signals the segmentation is complete.
When it does, it will stich the split images back together for each completed stage folder. When all stages are complete, it will finish executing.
If the cell is interrupted before completion, running it again will resume from where it left off.

In [8]:
def get_split_basenames(names):
    return [re.split("-[0-9]+\.",x)[0] for x in names]; #TODO: test if this is reliable for all filename structures

basenames_left = None;
# print(os.path.exists(completeMasks/'segmentation_complete.flag'));
while basenames_left is None or len(basenames_left) != 0 or not os.path.exists(completeMasks/'segmentation_complete.flag'):
    if os.path.exists(completeMasks/'segmentation_complete.flag'):
        print("full directory segmented and downloaded, completing stitching")
    else:
        print("Fetching changes from the cloud...")
        print(f'gsutil -o GSUtil:parallel_process_count=1 -m cp -r -n "gs://{(GCP_parent_dir/os.path.basename(completeMasks)).as_posix()}" "{processFolder}"')
        os.system(f'gsutil -o GSUtil:parallel_process_count=1 -m cp -r -n "gs://{(GCP_parent_dir/os.path.basename(completeMasks)).as_posix()}" "{processFolder}"');
    stitchedNames = [os.path.splitext(y)[0] for y in os.listdir(fullOutput)];
    unstitchedNames = os.listdir(completeMasks);
    basenames_left = [x for x in get_split_basenames(unstitchedNames) if x not in stitchedNames and not x.endswith((".nd",".flag"))];
    print(basenames_left[:30]);
    if basenames_left:
        print("new masks fetched: stitching");
    else:
        print("no new complete masks found, waiting for more");
    completed_names = [];
    print(len(basenames_left));
    print(len(set(basenames_left)))
    for b in tqdm(set(basenames_left)):
        if b in completed_names:
            continue;
        mask_pieces = [];
        error = False;
        for n in range(x_slices*y_slices):
            try:
                mask_pieces.append(imread(completeMasks/f"{b}-{n}.TIF"));
            except:
                error = True;
        if not error and len(mask_pieces) == 25:
            # print(f"image {b} ready for stitching");
            stitchMasks = [];
            # for piece in mask_pieces:
            #     try:
            #         stitchMasks.append(imread(completeMasks/piece));
            #     except Exception as e:
            #         print(f"error reading file",completeMasks/piece)
            for i,m in enumerate(mask_pieces):
                # print(i);
                x = i // y_slices;
                y = i % x_slices;
                # print(x,y);
                imBounds = [crop[0]+context_bounds[0] if x != 0 else 0,m.shape[0]-crop[2]-context_bounds[2] if x != x_slices-1 else m.shape[0],crop[1]+context_bounds[1] if y!= 0 else 0 ,m.shape[1]-crop[3]-context_bounds[3] if y != y_slices - 1 else m.shape[1]];
                stitchMasks.append(m[imBounds[0]:imBounds[1],imBounds[2]:imBounds[3]]);
                # print(stitchMasks[i].shape);
            stitched = np.concatenate([np.concatenate(stitchMasks[i*x_slices:(i+1)*x_slices],axis=1) for i in range(y_slices)]);
            # print(stitched.shape);
            imsave(fullOutput/(b+".TIF"),stitched,check_contrast=False);
            completed_names.append(b);
        elif os.path.exists(completeMasks/'segmentation_complete.flag'):
            print(f"error: unable to stitch mask {b}")

    for i in range(60):
        time.sleep(1); #allows interrupts during slompk
print("Image stitching fully complete");

full directory segmented and downloaded, completing stitching
[]
no new complete masks found, waiting for more
0
0


0it [00:00, ?it/s]


Image stitching fully complete


#### Clear temporary processing directories
Before running this cell, make sure that all of the files have been successfully transfered to the output directory movies/{movie_name}/full_segmentation

In [10]:
## Clear temporary directories
for im in tqdm(os.listdir(completeMasks)):
    f = os.path.join(completeMasks,im)
    if os.path.isdir(f): shutil.rmtree(f,onerror = on_rm_error ); #just in case
    else: os.remove(f);

100%|██████████| 86664/86664 [03:53<00:00, 371.76it/s]


## Full Segmentation

### Check if the full movie output directory is empty
If this directory is not empty, masks might be overwritten, and extra masks might be left in. Please copy any files you want to keep and then clear the directory.

In [31]:
##check if output directories empty:
if (any(os.scandir(fullOutput)) != 0):
    while True:
        doDelete = input(f"Warning: image processing directory {fullOutput}\nmust be empty; delete? (y/n), \'cancel\' to cancel\n",);
        if doDelete == "y":
            for im in os.scandir(fullOutput):
                try: shutil.rmtree(os.path.join(fullOutput,im),onerror = on_rm_error ); #just in case
                except OSError: os.remove(os.path.join(fullOutput,im));
            break;
        elif doDelete == "n":
            break;
        elif doDelete.lower() == "cancel":
            exit();

### Copy and split images from input directory to processing directory
Images will be split into subfolders based on their stage position so that segmentation can complete in chunks. This process requires all images to be in Metamorph output format.

In [35]:
##copy and split images from input directory to processing directory
names = os.listdir(moviePath);
print(len(names));
seen_stages = [];
stagedMaskNames = {};

def stage_from_name(name:str):
    m = re.match(filename_regex,name);
    return m.group(1) if m else "-1";

for k,g in itertools.groupby(names,stage_from_name):
    if k == "-1":
        continue;
    for name in list(g):
        if name.endswith('.nd'):
            print(f"ignoring .nd file");
            continue;
        match = re.match(filename_regex,name);
        if not match:
            print(f"Error: image {name} not valid filename, ignoring");
            continue;
        elif stage_ranges[int(match.group(1))] is not None and int(match.group(2)) not in stage_ranges[int(match.group(1))]:
            print(f"image {name} not in stage range, ignoring");
            continue;
        elif int(match.group(1)) not in seen_stages:
            if not os.path.exists(outImages/f"s{match.group(1)}"):
                os.mkdir(outImages/f"s{match.group(1)}");
            seen_stages.append(int(match.group(1)));
            stagedMaskNames[int(match.group(1))] = [];
        
        try:
            im = imread(moviePath/name);
        except Exception as e:
            print(e);
            continue;
        
        print(name);
        if auto_rescale:
            im = rescale_intensity(im);
        assert isinstance(im,np.ndarray);
        
        # print(im.shape);
        M = (im.shape[0]-context_bounds[0]-context_bounds[2]-crop[0]-crop[2])/y_slices;
        N = (im.shape[1]-context_bounds[1]-context_bounds[3]-crop[1]-crop[3])/x_slices;
        # print(M,N);
        # print(-context_bounds[1]-context_bounds[3]-crop[1]-crop[3])
        # print(im.shape);

        if int(M) != M or int(N) != N:
            raise Exception(f"ERROR: Mask with size {im.shape[:2]} cannot be sliced into {x_slices} columns and {y_slices} rows\nwith context bounds of {context_bounds}; {M} and {N} not integers");
        else:
            M = int(M)
            N = int(N)
            im = (im/256).astype('uint8');
            im = np.stack((im,im,im),axis=2);
            tiles = [im[y-context_bounds[0]:y+M+context_bounds[2],x-context_bounds[1]:x+N+context_bounds[3]] 
                    for y in range(context_bounds[0]+crop[0],im.shape[0]-crop[0]-crop[2]-context_bounds[0]-context_bounds[2],M) 
                    for x in range(context_bounds[1]+crop[1],im.shape[1]-crop[1]-crop[3]-context_bounds[1]-context_bounds[3],N)];
            imBounds = [[y-context_bounds[0],y+M+context_bounds[2],x-context_bounds[1],x+N+context_bounds[3]] 
                    for y in range(context_bounds[0]+crop[0],im.shape[0]-crop[0]-crop[2]-context_bounds[0]-context_bounds[2],M) 
                    for x in range(context_bounds[1]+crop[1],im.shape[1]-crop[1]-crop[3]-context_bounds[1]-context_bounds[3],N)];

            # print((context_bounds[0]+crop[0],context_bounds[1]+crop[1]))
            # print(imBounds);
            outMasks = [name,[]];
            for num,m in enumerate(tiles):
                outMasks[1].append(os.path.splitext(name)[0] + f"-{num}" + ".TIF");
                imsave(outImages/f"s{match.group(1)}"/(os.path.splitext(name)[0] + f"-{num}" + ".TIF"), m,check_contrast=False)
            stagedMaskNames[int(match.group(1))].append(outMasks.copy());
    print(f"stage {k} complete");
print("image transfer complete");

3857
p1_s10_t1.TIF
p1_s10_t10.TIF
p1_s10_t100.TIF
p1_s10_t101.TIF
p1_s10_t102.TIF
p1_s10_t103.TIF
p1_s10_t104.TIF
p1_s10_t105.TIF
p1_s10_t106.TIF
p1_s10_t107.TIF
p1_s10_t108.TIF
p1_s10_t109.TIF
p1_s10_t11.TIF
p1_s10_t110.TIF
p1_s10_t111.TIF
p1_s10_t112.TIF
p1_s10_t113.TIF
p1_s10_t114.TIF
p1_s10_t115.TIF
p1_s10_t116.TIF
p1_s10_t117.TIF
p1_s10_t118.TIF
p1_s10_t119.TIF
p1_s10_t12.TIF
p1_s10_t120.TIF
p1_s10_t121.TIF
p1_s10_t122.TIF
p1_s10_t123.TIF
p1_s10_t124.TIF
p1_s10_t125.TIF
p1_s10_t126.TIF
p1_s10_t127.TIF
p1_s10_t128.TIF
p1_s10_t129.TIF
p1_s10_t13.TIF
p1_s10_t130.TIF
p1_s10_t131.TIF
p1_s10_t132.TIF
p1_s10_t133.TIF
p1_s10_t134.TIF
p1_s10_t135.TIF
p1_s10_t136.TIF
p1_s10_t137.TIF
p1_s10_t138.TIF
p1_s10_t139.TIF
p1_s10_t14.TIF
p1_s10_t140.TIF
p1_s10_t141.TIF
p1_s10_t142.TIF
p1_s10_t143.TIF
p1_s10_t144.TIF
p1_s10_t145.TIF
p1_s10_t146.TIF
p1_s10_t147.TIF
p1_s10_t148.TIF
p1_s10_t149.TIF
p1_s10_t15.TIF
p1_s10_t150.TIF
p1_s10_t151.TIF
p1_s10_t152.TIF
p1_s10_t153.TIF
p1_s10_t154.TIF
p1_s10_t155

### Begin segmentation on remote hardware
if set-up correctly, google drive will start syncing files to the cloud while the above cell is processing. When you see "Stage {n} complete", begin the sementation

### Wait for remote segmentation to complete
This cell will constantly check each stage folder for a flag file from the remote segmentation program that signals the segmentation is complete.
When it does, it will stich the split images back together for each completed stage folder. When all stages are complete, it will finish executing.
If the cell is interrupted before completion, running it again will resume from where it left off.

In [88]:
stages_to_stitch = list(stagedMaskNames.keys()) if 'stages_to_stitch' not in locals() else stages_to_stitch;
print("stages to stitch:",stages_to_stitch)
while len(stages_to_stitch) != 0:
    stitched_stages = [];
    for stage in stages_to_stitch:
        if os.path.exists(completeMasks/f"s{stage}"/'segmentation_complete.flag') and not os.path.exists(completeMasks/f"s{stage}"/'stitching_complete.flag'):
            print(f"stage {stage} segmentation complete, waiting 60s for sync completion");
            time.sleep(60); #make sure the rest of the folder gets a chance to sync
            print("stitching stage...");
            ##output 
            # print(maskNames);
            success = True;
            for n in tqdm(stagedMaskNames[stage]):
                try:
                    if isinstance(n,list):
                        names = n[1];
                        baseName = n[0];
                        stitchMasks = [];
                        for name in names:
                            try:
                                stitchMasks.append(imread(completeMasks/f"s{stage}"/(os.path.splitext(name)[0] + maskSuffix + os.path.splitext(name)[1])));
                            except Exception as e:
                                print(f"error reading file",completeMasks/f"s{stage}"/(os.path.splitext(name)[0] + maskSuffix + os.path.splitext(name)[1]))
                        for i,m in enumerate(stitchMasks):
                            # print(i);
                            x = i // y_slices;
                            y = i % x_slices;
                            # print(x,y);
                            imBounds = [crop[0]+context_bounds[0] if x != 0 else 0,m.shape[0]-crop[2]-context_bounds[2] if x != x_slices-1 else m.shape[0],crop[1]+context_bounds[1] if y!= 0 else 0 ,m.shape[1]-crop[3]-context_bounds[3] if y != y_slices - 1 else m.shape[1]];
                            stitchMasks[i] = m[imBounds[0]:imBounds[1],imBounds[2]:imBounds[3]];
                            # print(stitchMasks[i].shape);
                        stitched = np.concatenate([np.concatenate(stitchMasks[i*x_slices:(i+1)*x_slices],axis=1) for i in range(y_slices)]);
                        # print(stitched.shape);
                        imsave(fullOutput/(os.path.splitext(baseName)[0]+".TIF"),stitched,check_contrast=False);
                except Exception as e:
                    print(f"error while splitting mask {n}:",e);
                    success = False;
            if success:
                with open(completeMasks/f"s{stage}"/'stitching_complete.flag','w') as f:
                    pass;
                stitched_stages.append(stage);
            else:
                print("error: image splitting not fully successful, not marking as complete");
            print("stitching complete!");
        else:
            if (os.path.exists(completeMasks/f"s{stage}"/'stitching_complete.flag')):
                stitched_stages.append(stage);
            time.sleep(10);
    [stages_to_stitch.remove(s) for s in stitched_stages];
    time.sleep(120);
            


stages to stitch: [10, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 10]
stage 6 segmentation complete, waiting 60s for sync completion
stitching stage...


100%|██████████| 199/199 [00:43<00:00,  4.60it/s]


stitching complete!
stage 7 segmentation complete, waiting 60s for sync completion
stitching stage...


100%|██████████| 241/241 [01:34<00:00,  2.55it/s]


stitching complete!
stage 8 segmentation complete, waiting 60s for sync completion
stitching stage...


100%|██████████| 241/241 [00:44<00:00,  5.39it/s]


stitching complete!
stage 9 segmentation complete, waiting 60s for sync completion
stitching stage...


100%|██████████| 241/241 [00:44<00:00,  5.38it/s]


stitching complete!


#### Clear temporary processing directories
Before running this cell, make sure that all of the files have been successfully transfered to the output directory movies/{movie_name}/full_segmentation

In [97]:
## Clear temporary directories
for im in os.scandir(outImages):
    try: shutil.rmtree(os.path.join(outImages,im),onerror = on_rm_error );
    except OSError: os.remove(os.path.join(outImages,im));
for im in os.scandir(completeMasks):
    try: shutil.rmtree(os.path.join(completeMasks,im),onerror = on_rm_error );
    except OSError: os.remove(os.path.join(completeMasks,im));